## Case Study
A US-based housing company named Surprise Housing has decided to enter the Australian market. The company uses data analytics to purchase houses at a price below their actual values and flip them on at a higher price. For the same purpose, the company has collected a data set from the sale of houses in Australia. The data is provided in the CSV file below.

The company is looking at prospective properties to buy to enter the market. You are required to build a regression model using regularisation in order to predict the actual value of the prospective properties and decide whether to invest in them or not.

 
#### Intent
The company wants to know:

- Which variables are significant in predicting the price of a house, and
- How well those variables describe the price of a house.
- Also, determine the optimal value of lambda for ridge and lasso regression.

#### Business Goal 

You are required to model the price of houses with the available independent variables. This model will then be used by the management to understand how exactly the prices vary with the variables. They can accordingly manipulate the strategy of the firm and concentrate on areas that will yield high returns. Further, the model will be a good way for management to understand the pricing dynamics of a new market.

In [ ]:
# Loading data 

df = pd.read_csv("./train.csv")

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
display(df)

## Data cleansing

In [ ]:
#Check null percentage
# print(df.isnull().sum())
def check_null(df):
    df_null = df.isnull().sum()
    print(df_null[df_null> 0])
    df_null = round(df_null[df_null> 0]/len(df.index), 9)
    df_null = df_null.sort_values()
    print(df_null)

In [ ]:
nd = check_null(df)
nd

In [ ]:
# Few categorical columns having missing values have business importance like House without Pool has PoolQC as null. Replace those values with a valid category value


df["BsmtQual"].fillna("NoBasement", inplace=True)
df["BsmtCond"].fillna("NoBasement", inplace=True)
df["BsmtFinType1"].fillna("NoBasement", inplace=True)
df["BsmtExposure"].fillna("NoBasement", inplace=True)
df["BsmtFinType2"].fillna("NoBasement", inplace=True)

df["GarageCond"].fillna("NoGarage", inplace=True)
df["GarageQual"].fillna("NoGarage", inplace=True)
df["GarageFinish"].fillna("NoGarage", inplace=True)
df["GarageType"].fillna("NoGarage", inplace=True)

df["FireplaceQu"].fillna("NoFireplace", inplace=True)
df["Fence"].fillna("NoFence", inplace=True)
df["Alley"].fillna("NoAlleyAccess", inplace=True)
df["MiscFeature"].fillna("None", inplace=True)
df["PoolQC"].fillna("NoPool", inplace=True)



In [ ]:
check_null(df)

In [ ]:
df.Electrical.mode()

In [ ]:
df['MasVnrType'].value_counts()

In [ ]:
df['MasVnrType'].fillna("None", inplace=True)

In [ ]:
# Data  Imputation
df['LotFrontage'].describe()

In [ ]:
df['MasVnrArea'].describe()

In [ ]:
df['GarageYrBlt'].describe()

In [ ]:
df['LotFrontage'] = df["LotFrontage"].transform(lambda x: x.fillna(df['LotFrontage'].median()))
df["GarageYrBlt"].fillna(df["GarageYrBlt"].median(), inplace=True)
df["MasVnrArea"].fillna(df["MasVnrArea"].median(), inplace=True)
df['Electrical'].fillna('SBrkr', inplace=True)


In [ ]:
#Identify the record where Basement value is specified where it is not having a basement for remaining basement columns
df1=df[(df.BsmtExposure=='NoBasement')]
df2=df[(df.BsmtFinType1=='NoBasement') & (df.BsmtCond=='NoBasement') & (df.BsmtQual=='NoBasement')]
df1[~df1.isin(df2)].dropna()

In [ ]:
#Identify the record where Basement value is specified where it is not having a basement for remaining basement columns
df1=df[(df.BsmtFinType2=='NoBasement')]
df2=df[(df.BsmtFinType1=='NoBasement') & (df.BsmtCond=='NoBasement') & (df.BsmtQual=='NoBasement')]
df1[~df1.isin(df2)].dropna()

In [ ]:
#deleting the records that are identified above and having bad data
df=df[(df.Id!=333.0) & (df.Id!=949.0)]

In [ ]:
check_null(df)

In [ ]:
df.dropna( inplace=True)
check_null(df)

### drop id columns as it has no business significance

In [ ]:
df.drop(columns="Id", inplace=True)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# we can see that all nulls has been handeled

# EDA
Performing EDA to understand more about columns

In [ ]:

def histogram(v):
    sns.distplot(v)

In [ ]:
histogram(df.GarageYrBlt)

In [ ]:
#getting the distribution plot for Garage built year. Based on this we see that 2000+ is the year where mostly built. 
#We can say any house built after 2000 as new built where as other as old

def EncodeGarageType(x):
    if x<2000:
        return 1
    if x>=2000:
        return 2
    if str(x)=='nan':
        return 0
df['GarageYrBlt']=df.GarageYrBlt.apply(EncodeGarageType)

In [ ]:

histogram(df['SalePrice'])

In [ ]:

print("Skewness =  %f" % df['SalePrice'].skew())
print("Kurtosis = %f" % df['SalePrice'].kurt())

### Observations

From the above distribution of saleprice the SalePrice is positively skewed  and to make it normally distributed we need to transform it to log scale
Since Kurtosis is greater than 3 , SalePrice contains outliers

In [ ]:
def desc(x):
    print(df[x].describe(percentiles=[.1,.25,.5,.75,.90,.95,0.97,0.98,0.99]))

In [ ]:
desc('SalePrice')

In [ ]:
# Since saleprice above 95% is highly skewed let's remove those out liers
quantile = df['SalePrice'].quantile(0.95)
df = df[df["SalePrice"] < quantile] 

In [ ]:
desc('SalePrice')

In [ ]:
df.describe(percentiles=[.1,.25,.5,.75,.90,.95,0.97,0.98,0.99])

In [ ]:
#Looks like LotArea has high skew between 99th and 100th percentile lets remove it
quantile = df['LotArea'].quantile(0.99)
df = df[df["LotArea"] < quantile] 

In [ ]:
#MiscVal there is a very high skew between 99th and 100 percentile lets remove those rows
quantile = df['MiscVal'].quantile(0.99)
df = df[df["MiscVal"] < quantile] 

In [ ]:
#MasVnrArea is nearly 3 time between 99th and 100th percentile lets remove those rows
quantile = df['MasVnrArea'].quantile(0.99)
df = df[df["MasVnrArea"] < quantile]

In [ ]:
#BsmtFinSF2 is twice beween 99th and 100 percentile lets remove it
quantile = df['BsmtFinSF2'].quantile(0.99)
df = df[df["BsmtFinSF2"] < quantile]

In [ ]:
plt.figure(figsize = (40, 20))
sns.heatmap(df.corr(), annot=True, cmap="jet")
plt.show()

In [ ]:
#there is 88% correlation between GarageCars and GarageArea so dropping cars
df.drop(columns="GarageCars", inplace=True)

#there is 76% corelation between TotalBsmtSF and 1stFlrSF dropping one
df.drop(columns="TotalBsmtSF", inplace=True)

#there is 83% corelation between TotRmsAbvGrd  and GrLivArea  dropping one
df.drop(columns="TotRmsAbvGrd", inplace=True)

In [ ]:
data_types = df.dtypes
numeric_cols = list(data_types[(data_types == 'int64') | (data_types == float)].index)
categorical_cols = list(data_types[data_types == object].index)
print(len(numeric_cols))
print(len(categorical_cols))

#### Observations
There are few variables like OverallQual, YearBuilt , YearRemodAdd, TotalBsmtSF, 1stFlrSF, GrLivArea, FullBath, GarageCars, GarageArea and MoSold are highly positvely correlated to SalePrice.


In [ ]:
# 'SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt'
#Lets plot the box plot to see the values
#set the number of columns to 2
numcol=2
#get the number of rows to plot
rows=len(numeric_cols)//numcol+1
#set the fig size
plt.figure(figsize=(17,32))
#for each numeric column plot
for index,column in enumerate(numeric_cols):
    #choose the plot
    plt.subplot(rows,numcol,index+1)
    #plot the box plot
    sns.boxplot(x=column, data=df, color='red')
plt.tight_layout()   
plt.show()  


#### Observations
We can see that data is nearly good but it is likely that we have some outliers. We cannot remove these as it will cause signifincat data loss

In [ ]:
# Check the numerical values using pairplots

plt.figure(figsize=(10,15))
sns.pairplot(df, x_vars=['MSSubClass', 'LotFrontage', 'LotArea'], y_vars='SalePrice',kind='scatter',diag_kind=None)
sns.pairplot(df, x_vars=['OverallQual', 'OverallCond', 'YearBuilt'], y_vars='SalePrice',kind='scatter',diag_kind=None)
sns.pairplot(df, x_vars=['YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1'], y_vars='SalePrice',kind='scatter',diag_kind=None)
sns.pairplot(df, x_vars=['BsmtFinSF2','BsmtUnfSF', '1stFlrSF'], y_vars='SalePrice',kind='scatter',diag_kind=None)
sns.pairplot(df, x_vars=['BsmtFullBath', 'FullBath'], y_vars='SalePrice',kind='scatter',diag_kind=None)
sns.pairplot(df, x_vars=['HalfBath', 'BedroomAbvGr', 'YrSold'], y_vars='SalePrice',kind='scatter',diag_kind=None)
sns.pairplot(df, x_vars=['Fireplaces', 'GarageYrBlt', 'GarageArea'], y_vars='SalePrice',kind='scatter',diag_kind=None)
sns.pairplot(df, x_vars=['WoodDeckSF','OpenPorchSF', 'EnclosedPorch'], y_vars='SalePrice',kind='scatter',diag_kind=None)
sns.pairplot(df, x_vars=['2ndFlrSF', 'GrLivArea'], y_vars='SalePrice',kind='scatter',diag_kind=None)
sns.pairplot(df, x_vars=[ 'FullBath'], y_vars='SalePrice',kind='scatter',diag_kind=None)

plt.show()

#### Observations
based on below plots we see some trend or linear pattern observed between SalesPrice and LotArea,YearBuilt, YearRemodAdd, BsmtFinSF1, 1stFlrSF, GarageArea, 2ndFlrSF,  GrLivArea, TotalBsmtSF

###  Data Preparation


In [ ]:
#### As we see earlier as part of EDA the SalePrice is not normally distributed and positively skewed let's transform the saleprice to log scale

In [ ]:
df["TransformedPrice"] = np.log(df["SalePrice"])

In [ ]:
sns.distplot(df["TransformedPrice"])

In [ ]:
#creating dummy columns for all string type categorical columsn
columns=df.select_dtypes(include=['object']).columns
for column in columns:
    df_d = pd.get_dummies(df[column], prefix=column, drop_first = True)
    df = pd.concat([df, df_d], axis = 1)
    df.drop(columns=[column],axis=1, inplace=True)
    

In [ ]:
df.head()

In [ ]:
df = df.drop(["SalePrice"], axis=1)
df.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler 


In [ ]:
df_train, df_test = train_test_split(df, train_size = 0.7, test_size = 0.3, random_state = 100)

In [ ]:

scaler = MinMaxScaler()

In [ ]:
#applying scaling on training set
num_vars = df_train.select_dtypes(include=np.number).columns
df_train[num_vars] = scaler.fit_transform(df_train[num_vars])
df_train.head()

In [ ]:
#applying the scaling learnt on Test set
df_test[num_vars] = scaler.transform(df_test[num_vars])
df_test.head()

In [ ]:
#generating X and Y sets for training and testing
y_train = df_train.pop('TransformedPrice')
X_train = df_train

y_test = df_test.pop('TransformedPrice')
X_test = df_test

### Model Building


In [ ]:
# list of alphas to tune
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}

In [ ]:
# Importing the relevant libraries
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model

In [ ]:
# Applying Lasso
lasso = Lasso()

# cross validation
folds = 5
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            

model_cv.fit(X_train, y_train) 

In [ ]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

In [ ]:
# Based on grid search best hyper parameters is 0.0001
print(model_cv.best_params_)
print(model_cv.best_score_)

In [ ]:
# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('float32')

# plotting
plt.figure(figsize=(16,5))
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('Negative Mean Absolute Error')

plt.title("Negative Mean Absolute Error and alpha")
plt.legend(['train score', 'test score'], loc='upper right')
plt.show()

In [ ]:
#taking alpha of 0.0001 and retraining model
alpha = 0.0001

lasso = Lasso(alpha=alpha)
        
lasso.fit(X_train, y_train)

In [ ]:
# Put the shortlisted Features and coefficienst in a dataframe

lasso_df = pd.DataFrame({'Features':X_train.columns, 'Coefficient':lasso.coef_.round(4)})
lasso_df = lasso_df[lasso_df['Coefficient'] != 0.00]
lasso_df.reset_index(drop=True, inplace=True)
lasso_df.sort_values('Coefficient', ascending=False).head(10).head(10)

In [ ]:
#getting the r2 value for lasso regression. It has score of 92.7% which is good
y_train_pred = lasso.predict(X_train)
r2_score(y_true=y_train, y_pred=y_train_pred)

In [ ]:
# Plotting y_test and y_pred to understand the spread.
fig = plt.figure()
plt.scatter(y_train,y_train_pred)
fig.suptitle('y_train vs y_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_train', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)                          # Y-label

In [ ]:
#plot the residuals. Based on plots the error are normally distributed
res=y_train-y_train_pred
sns.distplot(res)
plt.show()

In [ ]:
import statsmodels.api as sm

#plot the residuals
res=y_train-y_train_pred
sm.qqplot(res, line ='r')
plt.show()

In [ ]:
#r2 score on test data
y_test_pred=lasso.predict(X_test)
r2_score(y_test,y_test_pred)

In [ ]:
# Check the mean squared error

mean_squared_error(y_test, y_test_pred)

In [ ]:
# Put the Features and coefficienst in a dataframe
lasso_df = pd.DataFrame({'Features':X_train.columns, 'Coefficient':lasso.coef_.round(6)})
lasso_df.reset_index(drop=True, inplace=True)

In [ ]:
lasso_df.sort_values('Coefficient', ascending=False).head(10)

In [ ]:
#plot the residuals. Plot appears that residuals are normally distributed
res=y_test-y_test_pred
sns.distplot(res)
plt.show()

In [ ]:
res=y_test-y_test_pred
sm.qqplot(res, line ='r')
plt.show()

### Applying Ridge Linear Regression

In [ ]:
# Applying Ridge

ridge = Ridge()

# cross validation
folds = 5
#use the grid search to get the lambda value
model_cv = GridSearchCV(estimator = ridge, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            
model_cv.fit(X_train, y_train) 

In [ ]:

cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results = cv_results[cv_results['param_alpha']<=1000]
cv_results.head()

In [ ]:
#getting the best alpha value
print(model_cv.best_params_)
print(model_cv.best_score_)

In [ ]:
# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('int32')
plt.figure(figsize=(16,5))

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('Negative Mean Absolute Error')
plt.title("Negative Mean Absolute Error and alpha")
plt.legend(['train score', 'test score'], loc='upper right')
plt.show()

In [ ]:
#use alpha of 3.0
alpha = 3.0
ridge = Ridge(alpha=alpha)

ridge.fit(X_train, y_train)
#get the coefficients
ridge.coef_

In [ ]:
#getting the R2 score for training data. It seems that Ridge is gicving 92.6 R2 score on train set
y_train_pred=ridge.predict(X_train)
r2_score(y_train,y_train_pred)

In [ ]:
#getting the R2 score for training data. It seems that Ridge is gicving 88.1 R2 score on test
y_test_pred=ridge.predict(X_test)
r2_score(y_test,y_test_pred)

In [ ]:
# Check the mean squared error

mean_squared_error(y_test, y_test_pred)

In [ ]:
# Put the Features and coefficienst in a dataframe

ridge_df = pd.DataFrame({'Features':X_train.columns, 'Coefficient':ridge.coef_.round(6)})
ridge_df.reset_index(drop=True, inplace=True)

In [ ]:
ridge_df.sort_values('Coefficient', ascending=False).head(10)

In [ ]:
#plot the residuals
res=y_test-y_test_pred
sns.distplot(res)
plt.show()

In [ ]:
#plot the residuals
import statsmodels.api as sm
res=y_test-y_test_pred
sm.qqplot(res, line ='r')
plt.show()

## Conclusion

Based on 2 models that we have prediced Lasso seems to provide better accuracy.
Below are few conclusions of model
- lasso works best with alpha of 0.0001
- ridge works best with alpha of 3.0
- Lasso got higest r2 of 92.7 followed by ridge
- Based on Lasso following are the 10 key columns that influence the price of property

    GrLivArea	
    OverallCond	
    OverallQual	
    BsmtFinSF1	
    YearBuilt	
    GarageArea	
    BsmtUnfSF	
    LotArea
    MSZoning_RL	
    Neighborhood_Crawfor

### Questions

In [ ]:
#use alpha of 3.0
alpha = 3.0*2
ridge = Ridge(alpha=alpha)

ridge.fit(X_train, y_train)
#get the coefficients
ridge.coef_

#getting the R2 score for training data. It seems that Ridge is gicving 91.9 R2 score on train set
y_train_pred=ridge.predict(X_train)
r2_score(y_train,y_train_pred)

In [ ]:
ridge_df = pd.DataFrame({'Features':X_train.columns, 'Coefficient':ridge.coef_.round(6)})
ridge_df.reset_index(drop=True, inplace=True)

In [ ]:
x = ridge_df.sort_values('Coefficient', ascending=False).head(10)
x

In [ ]:

# Features	Coefficient
# 14	GrLivArea	0.141488
# 11	1stFlrSF	0.118750
# 3	OverallQual	0.113297
# 4	OverallCond	0.103395
# 8	BsmtFinSF1	0.084639
# 12	2ndFlrSF	0.077127
# 23	GarageArea	0.073562
# 10	BsmtUnfSF	0.072885
# 2	LotArea	0.059167
# 35	MSZoning_RL	0.046331

In [ ]:
x.Features

In [ ]:
alpha = 0.0001 * 2
lasso = Lasso(alpha=alpha)     
lasso.fit(X_train, y_train) 

In [ ]:
y_train_pred = lasso.predict(X_train)
r2_score(y_true=y_train, y_pred=y_train_pred)

In [ ]:
lasso_df = pd.DataFrame({'Features':X_train.columns, 'Coefficient':lasso.coef_.round(6)})
lasso_df.reset_index(drop=True, inplace=True)
lx = lasso_df.sort_values('Coefficient', ascending=False).head(10)
lx

In [ ]:
lx.Features

In [ ]:
len(lasso_df[lasso_df.Coefficient>0])

In [ ]:
len(ridge_df[ridge_df.Coefficient>0])

In [ ]:

X_train.drop(columns=['GrLivArea','OverallQual','OverallCond','BsmtFinSF1','YearBuilt'], inplace=True)

# Applying Lasso
lasso = Lasso()

# cross validation
folds = 5
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            

model_cv.fit(X_train, y_train) 

In [ ]:
# Based on grid search best hyper parameters is 0.0001
print(model_cv.best_params_)
print(model_cv.best_score_)

In [ ]:
alpha = 0.0001
lasso = Lasso(alpha=alpha)     
lasso.fit(X_train, y_train) 

In [ ]:
y_train_pred = lasso.predict(X_train)
r2_score(y_true=y_train, y_pred=y_train_pred)

In [ ]:
lasso_df = pd.DataFrame({'Features':X_train.columns, 'Coefficient':lasso.coef_.round(6)})
lasso_df.reset_index(drop=True, inplace=True)
lasso_df.sort_values('Coefficient', ascending=False).head(10)

In [ ]:
# 14	GrLivArea	0.353744
# 4	OverallCond	0.141184
# 3	OverallQual	0.137697
# 8	BsmtFinSF1	0.100313
# 5	YearBuilt	0.087424
# 23	GarageArea	0.081067
# 10	BsmtUnfSF	0.080736
# 2	LotArea	0.059988
# 35	MSZoning_RL	0.054618
# 58	Neighborhood_Crawfor	0.050446